In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision import models
from sklearn.metrics import accuracy_score
import cv2
import os
from tqdm import tqdm


In [ ]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
# Initialize the model
model = models.resnet18()

# Modify the final layer to match the number of classes (e.g., 1 for binary classification)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1)  # Adjust to match your output classes

# Load the trained weights
model = model.to(device)
model.load_state_dict(torch.load('resnet18_gender.pth'))
# model = model.to(device)
# Set the model to evaluation mode
model.eval()

<ipython-input-17-756fea5a9c5f>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('resnet18_gender.pth'))


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
from google.colab.patches import cv2_imshow
haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# image_folder_path = '/content/drive/My Drive/database_pics/'
image_folder_path = '/content/drive/My Drive/jihad/'

# Loop through each image in the folder
count = 0
for filename in os.listdir(image_folder_path):
    if filename.endswith(".jpg"):  # Only process jpg images
        image_path = os.path.join(image_folder_path, filename)


        image = cv2.imread(image_path)


        grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


        faces_rect = haar_cascade.detectMultiScale(grey, scaleFactor=1.1, minNeighbors=9)


        for (x, y, w, h) in faces_rect:
            cropped_face = image[y:y+h, x:x+w]


            cropped_face_rgb = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)
            input_tensor = transform(cropped_face_rgb)

            input_batch = input_tensor.unsqueeze(0)


            with torch.no_grad():
                output = model(input_batch)


            probability = torch.sigmoid(output).item()

            if probability > 0.5:
                predicted_class = "Female"
            else:
                predicted_class = "Male"
                probability = 1 - probability

            print(f"Image: {filename}, Predicted Class: {predicted_class}, Probability: {probability:.4f}")

            cv2_imshow(cropped_face)


In [ ]:
device

'cuda'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Image preprocessing with normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128)),  # Resize all images to 128x128
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

def get_limited_image_paths_and_labels(folder_path, label, limit):
    image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)[:limit]]
    labels = [label] * len(image_paths)
    return image_paths, labels


# Training data
image_folder_path = '/content/drive/My Drive/images/Training/'
male_folder_path = os.path.join(image_folder_path, 'male')
female_folder_path = os.path.join(image_folder_path, 'female')

# Take 1000 male and 100 female images for training
male_image_paths, male_labels = get_limited_image_paths_and_labels(male_folder_path, label=0, limit=2000)
female_image_paths, female_labels = get_limited_image_paths_and_labels(female_folder_path, label=1, limit=2000)

train_image_paths = male_image_paths + female_image_paths
train_labels = male_labels + female_labels

NameError: name 'transforms' is not defined

In [ ]:
validation_folder_path = '/content/drive/My Drive/images/Validation/'
male_val_folder = os.path.join(validation_folder_path, 'male')
female_val_folder = os.path.join(validation_folder_path, 'female')

# Take 250 male and 250 female images for validation
male_val_paths, male_val_labels = get_limited_image_paths_and_labels(male_val_folder, label=0, limit=500)
female_val_paths, female_val_labels = get_limited_image_paths_and_labels(female_val_folder, label=1, limit=500)

# Combine male and female validation data
val_image_paths = male_val_paths + female_val_paths
val_labels = male_val_labels + female_val_labels

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
      img_path = self.image_paths[idx]
      image = cv2.imread(img_path)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      if self.transform:
          image = self.transform(image)  # Convert to tensor
      label = torch.tensor(self.labels[idx], dtype=torch.float32)


      return image, label

# Create DataLoaders
train_dataset = ImageDataset(train_image_paths, train_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=8)

val_dataset = ImageDataset(val_image_paths, val_labels, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False,num_workers=8)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the final fully connected layer to output 1 (binary classification)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)  # Output is a single value for binary classification

# Move the model to the device
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 67.8MB/s]


In [ ]:
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0002)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []

    for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        all_preds.extend((outputs > 0.5).cpu().numpy())  # Convert outputs to binary predictions
        all_labels.extend(labels.cpu().numpy())

    train_loss = running_loss / len(train_loader)
    train_accuracy = accuracy_score(all_labels, all_preds)

    model.eval()
    val_loss = 0.0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_preds.extend((outputs > 0.5).cpu().numpy())  # Convert outputs to binary predictions
            val_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = accuracy_score(val_labels, val_preds)

    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
    print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

Validation Epoch 1/10:   0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Validation Epoch 1/10: 100%|██████████| 32/32 [01:07<00:00,  2.10s/it]


Epoch 1/10
Train Loss: 0.2144, Train Accuracy: 0.9110
Val Loss: 0.1191, Val Accuracy: 0.9600


Training Epoch 2/10:   0%|          | 0/125 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Validation Epoch 2/10:   0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potent

Epoch 2/10
Train Loss: 0.0627, Train Accuracy: 0.9780
Val Loss: 0.1328, Val Accuracy: 0.9490


Training Epoch 3/10:   0%|          | 0/125 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Validation Epoch 3/10:   0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potent

...

Epoch 8/10
Train Loss: 0.0203, Train Accuracy: 0.9938
Val Loss: 0.1564, Val Accuracy: 0.9590


Training Epoch 9/10:   0%|          | 0/125 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Validation Epoch 9/10:   0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potent

Epoch 9/10
Train Loss: 0.0187, Train Accuracy: 0.9930
Val Loss: 0.1336, Val Accuracy: 0.9600


Training Epoch 10/10:   0%|          | 0/125 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Validation Epoch 10/10:   0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid pote

Epoch 10/10
Train Loss: 0.0230, Train Accuracy: 0.9915
Val Loss: 0.1504, Val Accuracy: 0.9630


In [ ]:
# Save the fine-tuned model
torch.save(model.state_dict(), 'resnet18_gender_classifier.pth')
